In [1]:
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [2]:
data = pd.read_csv('../../input/clean_data/bici_clean.csv')

In [3]:
data.drop(['fecha', 'dia', 'year', 'tipo_accidente', 'distrito', 'direccion'],axis=1, inplace = True)

In [4]:
def num_horario(value):
    dict_hora = {
        'Mañana':1,
        'Tarde':2,
        'Noche':3
    }
    return dict_hora[value]

def num_fest(value):
    dict_fest = {
        'Laborable':1,
        'Vispera':2,
        'Festivo':3
    }
    return dict_fest[value]

In [5]:
data.horario = data.horario.apply(lambda x: num_horario(x))
data.festividad = data.festividad.apply(lambda x: num_fest(x))
data = pd.get_dummies(data, columns=['meteo'], prefix_sep='',prefix='')

In [6]:
data.head()

,horario,festividad,lesividad,lon,lat,Despejado,Lluvia,Niebla
0,2,3,Moderada,-3.681209,40.428990,1,0,0
1,3,1,Grave,-3.654527,40.466921,1,0,0
2,2,1,Leve,-3.623423,40.381971,1,0,0
3,2,1,Leve,-3.608276,40.377939,1,0,0
4,2,1,Leve,-3.608276,40.377939,1,0,0


In [7]:
X = data.drop(['lesividad'], axis=1)
y = data.lesividad
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [8]:
X.head()

,horario,festividad,lon,lat,Despejado,Lluvia,Niebla
0,2,3,-3.681209,40.428990,1,0,0
1,3,1,-3.654527,40.466921,1,0,0
2,2,1,-3.623423,40.381971,1,0,0
3,2,1,-3.608276,40.377939,1,0,0
4,2,1,-3.608276,40.377939,1,0,0


In [17]:
rfc = RandomForestClassifier(n_estimators=1000, n_jobs=-1)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)

params = rfc.get_params

dpre=pd.DataFrame({
    'pred':y_pred,
    'GT':y_test})

acierto = round((dpre[dpre.pred == dpre.GT].shape[0]/len(dpre.pred))*100,2)

with open('../../output/pred/records.txt', "a+") as file:
    file.write(
        f'''Model: RFC\t Acierto: {acierto}%\t Params: {params} \n\n'''
    )

In [8]:
knc = KNeighborsClassifier(n_jobs=-1)
knc.fit(X_train, y_train)
y_pred = knc.predict(X_test)

params = knc.get_params

dpre=pd.DataFrame({
    'pred':y_pred,
    'GT':y_test})

acierto = round((dpre[dpre.pred == dpre.GT].shape[0]/len(dpre.pred))*100,2)

with open('../../output/pred/records.txt', "a+") as file:
    file.write(
        f'''Model: KNC\t Acierto: {acierto}%\t Params: {params} \n\n'''
    )

In [9]:
from sklearn.externals import joblib 
  
# Save the model as a pickle in a file 
joblib.dump(knc, 'modelknc.pkl') 
  
# Load the model from the file 
# knn_from_joblib = joblib.load('filename.pkl')  
  
# # Use the loaded model to make predictions 
# knn_from_joblib.predict(X_test) 

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


['modelknc.pkl']

In [10]:
saved_model = pickle.dumps(knc) 